# Web Server Performance Analysis: Apache vs Nginx

## NSPNAO2025 Lab Week 05 - Performance Evaluation

This notebook performs comprehensive statistical analysis and visualization of Apache and Nginx web server performance under different resource constraints and concurrent user loads.

### Experimental Design:
- **Scenario A**: 0.5 CPU cores, 512MB RAM
- **Scenario B**: 1.0 CPU core, 512MB RAM
- **Load Levels**: 100, 500, 1000, 2500, 5000 VUs (Virtual Users)
- **Duration**: 120 seconds per test (30s ramp-up, 60s steady state, 30s ramp-down)

## 1. Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Data Loading Functions

In [2]:
def load_k6_csv(file_path):
    """
    Load k6 CSV results file.
    
    Args:
        file_path: Path to k6 CSV file
        
    Returns:
        DataFrame with k6 metrics
    """
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    return df

def parse_summary_file(file_path):
    """
    Parse k6 summary text file to extract key metrics.
    
    Args:
        file_path: Path to summary text file
        
    Returns:
        Dictionary with parsed metrics
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    metrics = {}
    
    # Extract http_req_duration metrics
    if 'http_req_duration' in content:
        line = [l for l in content.split('\n') if 'http_req_duration' in l and 'avg=' in l][0]
        parts = line.split()
        for part in parts:
            if 'avg=' in part:
                metrics['avg_duration'] = parse_duration(part.split('=')[1])
            elif 'p(95)=' in part:
                metrics['p95_duration'] = parse_duration(part.split('=')[1])
            elif 'p(90)=' in part:
                metrics['p90_duration'] = parse_duration(part.split('=')[1])
            elif 'max=' in part:
                metrics['max_duration'] = parse_duration(part.split('=')[1])
            elif 'min=' in part:
                metrics['min_duration'] = parse_duration(part.split('=')[1])
            elif 'med=' in part:
                metrics['med_duration'] = parse_duration(part.split('=')[1])
    
    # Extract http_reqs (throughput)
    if 'http_reqs' in content:
        line = [l for l in content.split('\n') if l.strip().startswith('http_reqs')][0]
        parts = line.split()
        if len(parts) >= 3:
            metrics['total_requests'] = int(parts[1])
            metrics['throughput'] = float(parts[2].replace('/s', ''))
    
    # Extract http_req_failed (error rate)
    if 'http_req_failed' in content:
        line = [l for l in content.split('\n') if l.strip().startswith('http_req_failed')][0]
        parts = line.split()
        if len(parts) >= 2:
            metrics['error_rate'] = float(parts[1].replace('%', '').replace(':', ''))
    
    # Extract checks
    if 'checks_succeeded' in content:
        line = [l for l in content.split('\n') if 'checks_succeeded' in l][0]
        parts = line.split()
        if len(parts) >= 2:
            metrics['checks_success_rate'] = float(parts[1].replace('%', '').replace(':', ''))
    
    return metrics

def parse_duration(duration_str):
    """
    Parse duration string from k6 output (e.g., '1.16ms', '2s', '31.29ms').
    
    Args:
        duration_str: Duration string
        
    Returns:
        Duration in milliseconds
    """
    duration_str = duration_str.strip()
    
    if 'ms' in duration_str:
        return float(duration_str.replace('ms', ''))
    elif 'µs' in duration_str or 'us' in duration_str:
        return float(duration_str.replace('µs', '').replace('us', '')) / 1000
    elif 'm' in duration_str:
        (minute, may_be_seconds) = duration_str.split("m")
        if not may_be_seconds:
            return float(minute) * 60
        return float(minute) * 60 + float(may_be_seconds[:-1])
    elif 's' in duration_str:
        return float(duration_str.replace('s', '')) * 1000
    elif 'µ' in duration_str:
        return float(duration_str.replace('µ', '')) / 1000
    else:
        return float(duration_str)

print("Data loading functions defined!")

Data loading functions defined!


## 3. Load All Test Results

In [3]:
# Define test parameters
results_dir = Path('results')
cpu_scenarios = ['cpu_0.5', 'cpu_1.0']
vu_levels = [100, 500, 1000, 2500, 5000]
servers = ['apache', 'nginx']

# Load all CSV data and calculate statistics from individual request measurements
all_results = []

for cpu_scenario in cpu_scenarios:
    cpu_limit = float(cpu_scenario.replace('cpu_', ''))
    
    for vus in vu_levels:
        vus_dir = results_dir / cpu_scenario / f'vus_{vus}'
        
        for server in servers:
            # Look for CSV files with pattern: {server}_{cpu}_{vus}.csv
            csv_pattern = f'{server}_{cpu_limit}_{vus}.csv'
            csv_file = vus_dir / csv_pattern
            
            if csv_file.exists():
                try:
                    # Load CSV file
                    df_raw = load_k6_csv(csv_file)
                    
                    # Extract http_req_duration measurements (individual request response times)
                    duration_data = df_raw[df_raw['metric_name'] == 'http_req_duration']

                    success_duration_data = duration_data[duration_data["status"] == 200]
                    
                    # Extract http_req_failed measurements (for error rate)
                    failed_data = df_raw[df_raw['metric_name'] == 'http_req_failed']
                    
                    # Calculate test duration (max timestamp - min timestamp)
                    test_duration = (df_raw['timestamp'].max() - df_raw['timestamp'].min()).total_seconds()

                    total_requests = len(duration_data['metric_value'])
                    failed_requests = len(failed_data['metric_value'])
                    
                    # Calculate statistics from raw measurements
                    if len(duration_data["metric_value"]) > 0:
                        metrics = {
                            'server': server,
                            'cpu_limit': cpu_limit,
                            'vus': vus,
                            'scenario': f'{cpu_limit} CPU',
                            'total_requests': total_requests,
                            "success_requests": total_requests - failed_requests,
                            "failed_requests": failed_requests,
                            # 'mean_duration': duration_data["metric_value"].mean(),
                            # 'std_duration': duration_data["metric_value"].std(),
                            "mean_success_requests": success_duration_data["metric_value"].mean(),                            
                            "std_success_requests": success_duration_data["metric_value"].std(),
                            # 'min_duration': duration_data.min(),
                            # 'max_duration': duration_data.max(),
                            # 'med_duration': duration_data.median(),
                            # 'p90_duration': duration_data.quantile(0.90),
                            # 'p95_duration': duration_data.quantile(0.95),
                            # 'p99_duration': duration_data.quantile(0.99),
                            # "success_requests": len()
                            # 'throughput': len(duration_data) / test_duration if test_duration > 0 else 0,
                            # 'error_rate': (failed_data.sum() / len(failed_data) * 100) if len(failed_data) > 0 else 0,
                        }
                        all_results.append(metrics)
                        
                except Exception as e:
                    print(f"Error loading {csv_file}: {e}")
            else:
                print(f"Warning: CSV file not found: {csv_file}")

# Create DataFrame
df_results = pd.DataFrame(all_results)

# Display the loaded data
print(f"\nLoaded {len(df_results)} test results from CSV files")
print(f"\nColumns: {', '.join(df_results.columns)}")
print(f"\nData shape: {df_results.shape}")
print(f"\nSample statistics (first test):")
if len(df_results) > 0:
    print(f"  - Requests measured: {df_results.iloc[0]['total_requests']:.0f}")
    # print(f"  - Avg duration: {df_results.iloc[0]['avg_duration']:.3f} ms (±{df_results.iloc[0]['std_duration']:.3f})")
    # print(f"  - P95 duration: {df_results.iloc[0]['p95_duration']:.3f} ms")
print()
df_results


Loaded 20 test results from CSV files

Columns: server, cpu_limit, vus, scenario, total_requests, success_requests, failed_requests, mean_success_requests, std_success_requests

Data shape: (20, 9)

Sample statistics (first test):
  - Requests measured: 7199



,server,cpu_limit,vus,scenario,total_requests,success_requests,failed_requests,mean_success_requests,std_success_requests
0,apache,0.5,100,0.5 CPU,7199,0,7199,1.164906,1.223432
1,nginx,0.5,100,0.5 CPU,7240,0,7240,2.166843,5.686595
2,apache,0.5,500,0.5 CPU,36283,0,36283,0.761393,1.165866
3,nginx,0.5,500,0.5 CPU,36203,0,36203,0.726490,1.242616
4,apache,0.5,1000,0.5 CPU,72397,0,72397,0.841698,1.668281
5,nginx,0.5,1000,0.5 CPU,72553,0,72553,0.802381,1.782607
6,apache,0.5,2500,0.5 CPU,102440,0,102440,377.709200,3747.335777
7,nginx,0.5,2500,0.5 CPU,181245,0,181245,0.871297,2.062455
8,apache,0.5,5000,0.5 CPU,112298,0,112298,674.009745,5167.208640
9,nginx,0.5,5000,0.5 CPU,357589,0,357589,15.966028,93.000230


## [คำตอบ] ขั้นตอนที่ 8: Design Experiment (ออกแบบและเตรียมการทดลอง)

### ตารางที่ 8.1: การทดสอบรอบที่ 1

บันทึกผลการทดสอบเมื่อจำกัด CPU ไว้ที่ 0.5 Core

In [4]:
server_sort = {'apache': 0, 'nginx': 1}

def format_answer_8(df: pd.DataFrame):
    return df.sort_values(
        by='server',
        key=lambda x: x.map(server_sort),
        ignore_index=True
    ).drop(columns=["cpu_limit"]).head(20)

In [5]:
format_answer_8(df_results[df_results["cpu_limit"] == 0.5])

,server,vus,scenario,total_requests,success_requests,failed_requests,mean_success_requests,std_success_requests
0,apache,100,0.5 CPU,7199,0,7199,1.164906,1.223432
1,apache,500,0.5 CPU,36283,0,36283,0.761393,1.165866
2,apache,2500,0.5 CPU,102440,0,102440,377.709200,3747.335777
3,apache,1000,0.5 CPU,72397,0,72397,0.841698,1.668281
4,apache,5000,0.5 CPU,112298,0,112298,674.009745,5167.208640
5,nginx,500,0.5 CPU,36203,0,36203,0.726490,1.242616
6,nginx,1000,0.5 CPU,72553,0,72553,0.802381,1.782607
7,nginx,100,0.5 CPU,7240,0,7240,2.166843,5.686595
8,nginx,2500,0.5 CPU,181245,0,181245,0.871297,2.062455
9,nginx,5000,0.5 CPU,357589,0,357589,15.966028,93.000230


### ตารางที่ 8.2: การทดสอบรอบที่ 1

บันทึกผลการทดสอบเมื่อจำกัด CPU ไว้ที่ 1.0 Core

In [6]:
format_answer_8(df_results[df_results["cpu_limit"] == 1.0])

,server,vus,scenario,total_requests,success_requests,failed_requests,mean_success_requests,std_success_requests
0,apache,100,1.0 CPU,7254,0,7254,1.280396,0.899930
1,apache,500,1.0 CPU,36242,0,36242,0.762299,1.102378
2,apache,2500,1.0 CPU,103240,0,103240,377.060163,3736.767337
3,apache,1000,1.0 CPU,72557,0,72557,0.723928,1.278048
4,apache,5000,1.0 CPU,112684,0,112684,668.235757,5213.868390
5,nginx,500,1.0 CPU,36289,0,36289,0.679066,0.901291
6,nginx,1000,1.0 CPU,72433,0,72433,0.717752,1.308848
7,nginx,100,1.0 CPU,7247,0,7247,1.223717,0.868286
8,nginx,2500,1.0 CPU,181037,0,181037,1.262209,3.945342
9,nginx,5000,1.0 CPU,361389,0,361389,4.108014,16.956417


## ขั้นตอนที่ 9: Analyze and Intepret Data (วิเคราะห์และแปลผลข้อมูล)

ใช้ค่าจากตารางผลทดลอง (เช่น Total Requests, Mean, Std Dev) ที่ได้จากขั้นตอนที่ 8 โดยในวิธีคำนวณของแลปกำหนดว่า กำหนดให้ใช้ **Z-distribution** (เพราะ n > 30) และใช้สูตร

$$
CI = \bar{x} \pm z_{1-\alpha/2} \left( \frac{s}{\sqrt{n}} \right)
$$

- $\bar{x}$ bar = Mean response time
- $n$ = Total Requests
- $s$ = Standard deviation estimate
- $z$ ที่ 95% = 1.96

> **สำคัญ:** ก่อนคำนวณให้ทำให้ “หน่วย" ของ Mean กับ Std Dev สอดคล้องกัน (แลปมี Mean เป็น ms และ Std Dev แสดงเป็น s ในตารางผลทดลอง) เพื่อไม่ให้ CI เพี้ยน

**Note 1!** ที่ความเชื่อมั่น 95% ค่า $\alpha$ = 0.05 เปิดหาตารางที่ $1 - \frac{\alpha}{2}$ = 0.975

**Note 2!** การคำนวณ Standard Distribution ของ **ประชากร** ให้ใช้สูตร

$$
s = \sqrt{\frac{\sum_i{|a_i - \bar{a}|^2 }}{N - 1}}
$$

*ตัว $-1$ ที่ใส่เพิ่มไปนี้ อาจหมายถึง **Degree of Freedom** ก็ได้*

**Note 3!** ทั้งก้อนหลัง $\bar{x} \pm$ อาจเรียกอีกชื่อว่า **Margin of Error**

In [9]:
confidence = 0.95
z_critical = stats.norm.ppf((1 + confidence) / 2)

print("z =", z_critical)

z = 1.959963984540054


### 9.1 รวบรวมข้อมูลดิบ (Data Collection)

สรุปผลลัพธ์ที่ได้จากการทดลองทั้ง 10 รอบลงในตารางนี้ เพื่อให้เห็นภาพรวมเปรียบเทียบกันชัดเจน

**ตาราง 9.1: CI Calculation Results — CPU Limit = 0.5 Core**

In [79]:
def calculate_confidence_interval(df, confidence=0.95):
    # ค่า std, n ใช้จากที่มีอยู่แล้ว
    n = df["total_requests"]
    std = df["std_success_requests"]
    mean = df["mean_success_requests"]
    
    margin_of_error = z_critical * (std / np.sqrt(n))

    # Confidence Interval คือ ช่วง ดังนั้นจะมีช่วงเปิดและปิด
    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error

    return mean, lower_bound, upper_bound, margin_of_error


def format_answer_9(df: pd.DataFrame):
    # เรียงคอลัมน์ใหม่
    df_out = df[["vus", "server", "mean_success_requests", "total_requests", "std_success_requests"]].reset_index(drop=True)
    
    """
    Goal 1: Calculate 95% CI
    """

    mean, lower_bound, upper_bound, margin_of_error = calculate_confidence_interval(df.reset_index(drop=True))  

    # ไม่ได้ใช้ตอบ แปะให้เห็นในตารางเฉย ๆ
    df_out["margin_of_error"] = margin_of_error

    # Static keys. Too lazy to type again and again
    LOWER = "0.95_ci_lower_bound"
    UPPER = "0.95_ci_upper_bound"

    df_out[LOWER] = lower_bound
    df_out[UPPER] = upper_bound

    """
    Goal 2: Find overlapping in Confidence Interval
    """
    # สังเกตจาก Pattern ของข้อมูลว่า Apache จะมาก่อน Nginx และจะมาคู่กัน
    # คำนวณทุก ๆ ครั้งที่เจอ Nginx (aka เริีมที่ 1 จบที่อันสุดท้าย เพิ่มทีละ 2 จ้า)
    for i in range(1, len(df), 2):
        apache = df_out.iloc[i - 1]
        nginx = df_out.iloc[i]
        # Check overlap
        if apache[UPPER] >= nginx[LOWER] and nginx[UPPER] >= apache[LOWER]:
            df_out.loc[i-1:i, "is_overlapping"] = "overlap"
            df_out.loc[i-1:i, "delta_overlap"] = 0.
        
        # No overlap: determine which is better
        # Because we calculating response time, lower is considered better
        elif nginx[LOWER] > apache[UPPER]:
            df_out.loc[i-1:i, "is_overlapping"] = "apache_better"
            df_out.loc[i-1:i, "delta_overlap"] = (nginx[LOWER] - apache[UPPER])
        else:
            df_out.loc[i-1:i, "is_overlapping"] = "nginx_better"
            df_out.loc[i-1:i, "delta_overlap"] = apache[LOWER] - nginx[UPPER]
    return df_out


format_answer_9(df_results[df_results["cpu_limit"] == 0.5])

,vus,server,mean_success_requests,total_requests,std_success_requests,margin_of_error,0.95_ci_lower_bound,0.95_ci_upper_bound,is_overlapping,delta_overlap
0,100,apache,1.164906,7199,1.223432,0.028261,1.136645,1.193167,apache_better,0.842688
1,100,nginx,2.166843,7240,5.686595,0.130988,2.035855,2.297831,apache_better,0.842688
2,500,apache,0.761393,36283,1.165866,0.011996,0.749397,0.773390,nginx_better,0.010107
3,500,nginx,0.726490,36203,1.242616,0.012800,0.713690,0.739290,nginx_better,0.010107
4,1000,apache,0.841698,72397,1.668281,0.012152,0.829546,0.853850,nginx_better,0.014194
5,1000,nginx,0.802381,72553,1.782607,0.012971,0.789410,0.815352,nginx_better,0.014194
6,2500,apache,377.709200,102440,3747.335777,22.947528,354.761672,400.656729,nginx_better,353.880880
7,2500,nginx,0.871297,181245,2.062455,0.009495,0.861802,0.880792,nginx_better,353.880880
8,5000,apache,674.009745,112298,5167.208640,30.221643,643.788101,704.231388,nginx_better,627.517256
9,5000,nginx,15.966028,357589,93.000230,0.304818,15.661210,16.270845,nginx_better,627.517256


In [80]:
format_answer_9(df_results[df_results["cpu_limit"] == 1.0])

,vus,server,mean_success_requests,total_requests,std_success_requests,margin_of_error,0.95_ci_lower_bound,0.95_ci_upper_bound,is_overlapping,delta_overlap
0,100,apache,1.280396,7254,0.899930,0.020709,1.259686,1.301105,nginx_better,0.015979
1,100,nginx,1.223717,7247,0.868286,0.019991,1.203726,1.243708,nginx_better,0.015979
2,500,apache,0.762299,36242,1.102378,0.011349,0.750949,0.773648,nginx_better,0.062610
3,500,nginx,0.679066,36289,0.901291,0.009273,0.669793,0.688339,nginx_better,0.062610
4,1000,apache,0.723928,72557,1.278048,0.009299,0.714629,0.733228,overlap,0.000000
5,1000,nginx,0.717752,72433,1.308848,0.009532,0.708220,0.727284,overlap,0.000000
6,2500,apache,377.060163,103240,3736.767337,22.793979,354.266183,399.854142,nginx_better,352.985800
7,2500,nginx,1.262209,181037,3.945342,0.018174,1.244035,1.280383,nginx_better,352.985800
8,5000,apache,668.235757,112684,5213.868390,30.442270,637.793487,698.678027,nginx_better,633.630190
9,5000,nginx,4.108014,361389,16.956417,0.055283,4.052731,4.163297,nginx_better,633.630190


## 9. Statistical Significance Testing with Confidence Intervals

## 5. Capacity Curves: Throughput vs Load

In [ ]:
def plot_capacity_curve_throughput(df, cpu_limit):
    """
    Plot throughput vs VUs (load) for capacity analysis.
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    scenario_data = df[df['cpu_limit'] == cpu_limit]
    
    for server in ['apache', 'nginx']:
        server_data = scenario_data[scenario_data['server'] == server].sort_values('vus')
        
        x = server_data['vus']
        y = server_data['throughput_mean']
        
        # Plot line
        marker = 'o' if server == 'apache' else 's'
        ax.plot(x, y, marker=marker, linewidth=2, markersize=8, label=server.capitalize())
        
        # Add error bars if std is available
        if 'throughput_std' in server_data.columns:
            yerr = server_data['throughput_std']
            ax.errorbar(x, y, yerr=yerr, fmt='none', alpha=0.3, capsize=5)
    
    ax.set_xlabel('Virtual Users (VUs)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Throughput (requests/sec)', fontsize=12, fontweight='bold')
    ax.set_title(f'Capacity Curve: Throughput vs Load ({cpu_limit} CPU)', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot for both scenarios
for cpu in [0.5, 1.0]:
    plot_capacity_curve_throughput(df_summary, cpu)
    plt.show()

## 6. Capacity Curves: Response Time vs Load

In [ ]:
def plot_capacity_curve_response_time(df, cpu_limit, response_time_limit=500):
    """
    Plot response time (p95) vs VUs with acceptable threshold line.
    
    Args:
        df: DataFrame with results
        cpu_limit: CPU limit to filter
        response_time_limit: Acceptable p95 response time threshold in ms
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    scenario_data = df[df['cpu_limit'] == cpu_limit]
    
    for server in ['apache', 'nginx']:
        server_data = scenario_data[scenario_data['server'] == server].sort_values('vus')
        
        x = server_data['vus']
        y = server_data['p95_duration_mean']
        
        # Plot line
        marker = 'o' if server == 'apache' else 's'
        ax.plot(x, y, marker=marker, linewidth=2, markersize=8, label=f'{server.capitalize()} (p95)')
        
        # Add error bars if std is available
        if 'p95_duration_std' in server_data.columns:
            yerr = server_data['p95_duration_std']
            ax.errorbar(x, y, yerr=yerr, fmt='none', alpha=0.3, capsize=5)
    
    # Add acceptable threshold line
    ax.axhline(y=response_time_limit, color='red', linestyle='--', linewidth=2, 
               label=f'Acceptable Limit ({response_time_limit}ms)', alpha=0.7)
    
    ax.set_xlabel('Virtual Users (VUs)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Response Time p95 (ms)', fontsize=12, fontweight='bold')
    ax.set_title(f'Capacity Curve: Response Time vs Load ({cpu_limit} CPU)', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot for both scenarios
for cpu in [0.5, 1.0]:
    plot_capacity_curve_response_time(df_summary, cpu, response_time_limit=500)
    plt.show()

## 7. Knee Point Detection and Usable Capacity

In [ ]:
def find_knee_point(df, server, cpu_limit, response_time_limit=500):
    """
    Find the knee point where throughput plateaus and response time exceeds threshold.
    
    Returns:
        Dictionary with knee point analysis
    """
    server_data = df[(df['server'] == server) & (df['cpu_limit'] == cpu_limit)].sort_values('vus')
    
    if len(server_data) == 0:
        return None
    
    # Find where p95 exceeds threshold
    exceeds_threshold = server_data[server_data['p95_duration_mean'] > response_time_limit]
    
    if len(exceeds_threshold) > 0:
        knee_vus = exceeds_threshold.iloc[0]['vus']
        # Usable capacity is the VU level just before knee point
        usable_capacity_idx = server_data[server_data['vus'] < knee_vus].index
        if len(usable_capacity_idx) > 0:
            usable_capacity = server_data.loc[usable_capacity_idx[-1]]
        else:
            usable_capacity = None
    else:
        knee_vus = None
        usable_capacity = server_data.iloc[-1]  # All loads are acceptable
    
    # Calculate throughput efficiency (throughput per VU)
    server_data['efficiency'] = server_data['throughput_mean'] / server_data['vus']
    
    result = {
        'server': server,
        'cpu_limit': cpu_limit,
        'knee_point_vus': knee_vus,
        'max_throughput': server_data['throughput_mean'].max(),
        'peak_efficiency': server_data['efficiency'].max(),
    }
    
    if usable_capacity is not None:
        result.update({
            'usable_capacity_vus': usable_capacity['vus'],
            'usable_throughput': usable_capacity['throughput_mean'],
            'usable_p95_response': usable_capacity['p95_duration_mean'],
        })
    
    return result

# Find knee points for all configurations
knee_points = []
for cpu in [0.5, 1.0]:
    for server in ['apache', 'nginx']:
        kp = find_knee_point(df_summary, server, cpu, response_time_limit=500)
        if kp:
            knee_points.append(kp)

df_knee_points = pd.DataFrame(knee_points)

print("\n" + "="*80)
print("KNEE POINT ANALYSIS & USABLE CAPACITY")
print("="*80)
df_knee_points

## 8. Side-by-Side Comparison: Apache vs Nginx

In [ ]:
def plot_server_comparison(df, cpu_limit, metric='throughput_mean', ylabel='Throughput (req/s)'):
    """
    Create side-by-side bar chart comparing Apache and Nginx.
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    scenario_data = df[df['cpu_limit'] == cpu_limit]
    
    x = np.arange(len(vu_levels))
    width = 0.35
    
    apache_data = scenario_data[scenario_data['server'] == 'apache'].sort_values('vus')
    nginx_data = scenario_data[scenario_data['server'] == 'nginx'].sort_values('vus')
    
    bars1 = ax.bar(x - width/2, apache_data[metric], width, label='Apache', alpha=0.8)
    bars2 = ax.bar(x + width/2, nginx_data[metric], width, label='Nginx', alpha=0.8)
    
    ax.set_xlabel('Virtual Users (VUs)', fontsize=12, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=12, fontweight='bold')
    ax.set_title(f'Performance Comparison: {ylabel} ({cpu_limit} CPU)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(vu_levels)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.1f}',
                       xy=(bar.get_x() + bar.get_width() / 2, height),
                       xytext=(0, 3),
                       textcoords="offset points",
                       ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    return fig

# Create comparison charts
for cpu in [0.5, 1.0]:
    print(f"\n{'='*80}")
    print(f"SCENARIO: {cpu} CPU")
    print(f"{'='*80}")
    
    plot_server_comparison(df_summary, cpu, 'throughput_mean', 'Throughput (req/s)')
    plt.show()
    
    plot_server_comparison(df_summary, cpu, 'p95_duration_mean', 'Response Time p95 (ms)')
    plt.show()
    
    plot_server_comparison(df_summary, cpu, 'error_rate_mean', 'Error Rate (%)')
    plt.show()

In [44]:
def plot_confidence_intervals(df, cpu_limit, metric='throughput'):
    """
    Plot confidence intervals for visual hypothesis testing.
    Overlapping CIs suggest no significant difference.
    """
    fig, ax = plt.subplots(figsize=(14, 7))
    
    scenario_data = df_results[(df_results['cpu_limit'] == cpu_limit)]
    
    y_pos = 0
    yticks = []
    yticklabels = []
    
    for vus in vu_levels:
        for server in ['apache', 'nginx']:
            data = scenario_data[(scenario_data['vus'] == vus) & (scenario_data['server'] == server)][metric]
            
            if len(data) > 0:
                mean, lower, upper, moe = calculate_confidence_interval(data.values)
                
                color = 'blue' if server == 'apache' else 'green'
                ax.errorbar(mean, y_pos, xerr=[[mean-lower], [upper-mean]], 
                           fmt='o', color=color, capsize=5, capthick=2, markersize=8)
                
                yticks.append(y_pos)
                yticklabels.append(f"{server.capitalize()}\n{vus} VUs")
                y_pos += 1
        
        y_pos += 0.5  # Add spacing between VU groups
    
    ax.set_yticks(yticks)
    ax.set_yticklabels(yticklabels)
    ax.set_xlabel(f'{metric.replace("_", " ").title()}', fontsize=12, fontweight='bold')
    ax.set_title(f'95% Confidence Intervals: {metric.replace("_", " ").title()} ({cpu_limit} CPU)', 
                fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='blue', label='Apache'),
                      Patch(facecolor='green', label='Nginx')]
    ax.legend(handles=legend_elements, fontsize=11)
    
    plt.tight_layout()
    return fig

# Plot confidence intervals for key metrics
for cpu in [0.5, 1.0]:
    print(f"\n{'='*80}")
    print(f"95% Confidence Intervals - {cpu} CPU")
    print(f"{'='*80}")
    
    if 'throughput' in df_results.columns:
        plot_confidence_intervals(df_results, cpu, 'throughput')
        plt.show()
    
    if 'p95_duration' in df_results.columns:
        plot_confidence_intervals(df_results, cpu, 'p95_duration')
        plt.show()


95% Confidence Intervals - 0.5 CPU

95% Confidence Intervals - 1.0 CPU


## 9a. Visual Hypothesis Testing: Mean Response Time with Error Bars

This visualization shows the mean response time for Apache and Nginx servers across different load levels, with error bars representing the 95% confidence interval of the mean. Non-overlapping error bars suggest statistically significant differences between the servers at that load level.

In [ ]:
def plot_mean_response_time_with_error_bars():
    """
    Create error bar plots for mean response time comparison between Apache and Nginx.
    Shows 95% confidence interval of the mean for visual hypothesis testing.
    
    Error bars represent: mean ± 1.96 * (std / sqrt(n))
    where n is the number of requests measured.
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    cpu_scenarios = [
        (0.5, 'Visual Hypothesis Testing: CPU 0.5 Core (High Load)'),
        (1.0, 'Visual Hypothesis Testing: CPU 1 Core (High Load)')
    ]
    
    for idx, (cpu_limit, title) in enumerate(cpu_scenarios):
        ax = axes[idx]
        
        # Filter data for this CPU scenario
        scenario_data = df_results[df_results['cpu_limit'] == cpu_limit].sort_values(['server', 'vus'])
        
        # Separate data for each server
        apache_data = scenario_data[scenario_data['server'] == 'apache'].sort_values('vus')
        nginx_data = scenario_data[scenario_data['server'] == 'nginx'].sort_values('vus')
        
        # Calculate 95% confidence interval for mean
        # CI = mean ± 1.96 * (std / sqrt(n))
        # Using z-value for 95% CI
        z_value = 1.96
        
        # Apache calculations
        apache_vus = apache_data['vus'].values
        apache_mean = apache_data['avg_duration'].values
        apache_std = apache_data['std_duration'].values
        apache_n = apache_data['total_requests'].values
        apache_ci = z_value * (apache_std / np.sqrt(apache_n))
        
        # Nginx calculations
        nginx_vus = nginx_data['vus'].values
        nginx_mean = nginx_data['avg_duration'].values
        nginx_std = nginx_data['std_duration'].values
        nginx_n = nginx_data['total_requests'].values
        nginx_ci = z_value * (nginx_std / np.sqrt(nginx_n))

        print(apache_vus, apache_mean, apache_ci)
        print(nginx_vus, nginx_mean, nginx_ci)
        
        # Plot error bars
        ax.errorbar(apache_vus, apache_mean, yerr=apache_ci, 
                   fmt='o', color='red', markersize=8, capsize=5, capthick=2,
                   label='Apache', linewidth=2, elinewidth=2)
        
        ax.errorbar(nginx_vus, nginx_mean, yerr=nginx_ci,
                   fmt='s', color='green', markersize=8, capsize=5, capthick=2,
                   label='Nginx', linewidth=2, elinewidth=2)
        
        # Styling
        ax.set_xlabel('Load (VUs)', fontsize=11, fontweight='bold')
        ax.set_ylabel('Mean Response Time (ms)', fontsize=11, fontweight='bold')
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.legend(loc='upper_bound left', fontsize=10)
        ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
        ax.set_facecolor('white')
        
        # Set x-axis ticks
        ax.set_xticks(vu_levels)
        ax.set_xticklabels(vu_levels)
        
        # Set y-axis to start from 0 for better comparison
        # But adjust if values are too large (like at 5000 VUs)
        max_val = max(apache_mean[-1] + apache_ci[-1], nginx_mean[-1] + nginx_ci[-1])
        if max_val > 1000:
            # Use log scale for better visualization when values are very large
            ax.set_yscale('log')
            ax.set_ylabel('Mean Response Time (ms) - Log Scale', fontsize=11, fontweight='bold')
        else:
            ax.set_ylim(bottom=0)
    
    plt.tight_layout()
    return fig

# Create the visualization
print("\n" + "="*100)
print("VISUAL HYPOTHESIS TESTING: MEAN RESPONSE TIME")
print("="*100)
print("\nError bars represent 95% confidence interval of the mean (±1.96 × SE)")
print("SE (Standard Error) = std / sqrt(n), where n is the number of requests measured")
print("\nInterpretation:")
print("  - Non-overlapping error bars → Statistically significant difference")
print("  - Overlapping error bars → No significant difference at 95% confidence level")
print("="*100)

plot_mean_response_time_with_error_bars()
plt.show()

## 10. Performance Summary Table

In [ ]:
def create_summary_table(df):
    """
    Create a comprehensive summary table.
    """
    summary = df.pivot_table(
        index=['cpu_limit', 'vus'],
        columns='server',
        values=['throughput_mean', 'p95_duration_mean', 'error_rate_mean'],
        aggfunc='first'
    )
    
    return summary

summary_table = create_summary_table(df_summary)

print("\n" + "="*100)
print("COMPREHENSIVE PERFORMANCE SUMMARY")
print("="*100)
print("\nThroughput (req/s) | Response Time p95 (ms) | Error Rate (%)")
print("="*100)
summary_table

## 11. Key Findings and Recommendations

In [ ]:
print("\n" + "="*100)
print("KEY FINDINGS")
print("="*100)

for cpu in [0.5, 1.0]:
    print(f"\n### Scenario: {cpu} CPU Core(s)\n")
    
    scenario_knee = df_knee_points[df_knee_points['cpu_limit'] == cpu]
    
    for _, row in scenario_knee.iterrows():
        server = row['server'].capitalize()
        print(f"**{server}:**")
        print(f"  - Maximum Throughput: {row['max_throughput']:.2f} req/s")
        
        if pd.notna(row.get('knee_point_vus')):
            print(f"  - Knee Point: {row['knee_point_vus']} VUs")
        else:
            print(f"  - Knee Point: Not reached (response time acceptable at all loads)")
        
        if pd.notna(row.get('usable_capacity_vus')):
            print(f"  - Usable Capacity: {row['usable_capacity_vus']} VUs")
            print(f"  - Usable Throughput: {row['usable_throughput']:.2f} req/s")
            print(f"  - Response Time at Capacity: {row['usable_p95_response']:.2f} ms (p95)")
        
        print()

print("\n" + "="*100)
print("RECOMMENDATIONS")
print("="*100)
print("""
1. **Resource Allocation**: Compare the usable capacity between 0.5 and 1.0 CPU scenarios
   to determine the cost-benefit of additional resources.

2. **Server Selection**: Choose the server (Apache vs Nginx) that provides better throughput
   and lower response times at your expected load level.

3. **Capacity Planning**: Deploy enough instances to stay below the knee point with headroom
   for traffic spikes (typically 60-70% of usable capacity).

4. **Performance Monitoring**: Set alerts when p95 response time exceeds 400ms (80% of 500ms
   threshold) to proactively scale before hitting capacity limits.

5. **Load Testing**: Regularly repeat these tests as application code changes, as even small
   modifications can significantly impact performance characteristics.
""")

print("\n" + "="*100)
print("ANALYSIS COMPLETE")
print("="*100)

## 12. Export Results for Further Analysis

In [ ]:
# Export summary tables to CSV
df_summary.to_csv('performance_summary.csv', index=False)
df_knee_points.to_csv('knee_points_analysis.csv', index=False)

print("\nResults exported:")
print("  - performance_summary.csv")
print("  - knee_points_analysis.csv")
print("\nAnalysis complete! Review the visualizations and tables above.")